<a href="https://colab.research.google.com/github/npr99/Archive/blob/master/PLAN604_ACS_Population_Pyramids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of Population Pyramids from ACS Data
 
---
This Google Colab Notebook provides a complete workflow (sequence of steps from start to finish) that will allow you to explore population composition data by age and sex found in ACS Data. 

This notebook obtains and cleans population percentages found in the ACS.


Helpful Links that inspired this notebook:

https://www.youtube.com/watch?v=jMBaY-rO4G0

https://walker-data.com/tidycensus/articles/other-datasets.html#migration-flows-1


In [ ]:
# Python packages required to read in and Census API data
import requests ## Required for the Census API
import pandas as pd # For reading, writing and wrangling data
import matplotlib.pyplot as plt # For plotting and making graphs
import numpy as np

In [50]:
# Downloading and running python script from github
# https://jckantor.github.io/cbe61622/A.02-Downloading_Python_source_files_from_github.html
# Make sure the url is the raw version of the file on GitHub

user = "npr99"
repo = "PlanningMethods"
pyfile = "_planning_methods.py"
url = f"https://raw.githubusercontent.com/{user}/{repo}/master/{pyfile}"
!wget --no-cache --quiet --backups=1 {url}
print("Reading in python file from",url)
exec(open(pyfile).read())

Reading in python file from https://raw.githubusercontent.com/npr99/PlanningMethods/master/_planning_methods.py


## Step 1: Obtain Data
The previous step read in a python file with the Obtain Data function presented in the notebook on [Sample Size and Confidence Intervals](https://github.com/npr99/PlanningMethods/blob/master/PLAN604_Population_vs_Sample_USCounties.ipynb) The block of code with the python functions needs to be run first and then the function can be called in future blocks of code.

## Run Obtain Census API for ACS data with Age and Sex Charactersitics
The next block of code calls the function and gets varaibles related to public transportation use. The code is called in a loop.

For more variables see:

https://www.census.gov/data/developers/data-sets/acs-5year.2019.html

https://api.census.gov/data/2019/acs/acs5/subject/groups/S0101.html

https://data.census.gov/cedsci/table?tid=ACSST1Y2019.S0101


### Use a loop to make list of variables to get

In [51]:
tableid = 'S0101'
columns = {4 : 'Percent Male',
           6 : 'Percent Female'
           }
# There are 18 age cohorts under 5 years to 85 years and over
age_cohorts = 18
# What is the variable number for thefirst age cohort
first_age_cohort_variable = 2

# Create an empty "container" to store multiple ACS years for the data
acs_df = {} 

dataset_name = 'acs/acs5/subject'
vintages = ['2019']

for column in columns:
  get_vars = 'GEO_ID,NAME'
  for age_variable in range(0,age_cohorts):
    age_cohort_var = first_age_cohort_variable + age_variable
    age_cohort_var_str = str(age_cohort_var).zfill(3)
    get_var = tableid+'_C0'+str(column)+"_"+age_cohort_var_str+'E'
    get_vars = get_vars + ',' + get_var

  # Get dat for the list of variables
  for vintage in vintages:
    acs_df[vintage+' '+columns[column]] = planning_methods.obtain_census_api(get_vars = get_vars,
                                                        dataset_name = dataset_name, 
                                                        vintage = vintage)
    
    acs_df[vintage+' '+columns[column]] = clean_acs_variables(acs_df[vintage+' '+columns[column]],vintage, dataset_name, get_vars)
        



Census API data from: https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S0101_C04_002E,S0101_C04_003E,S0101_C04_004E,S0101_C04_005E,S0101_C04_006E,S0101_C04_007E,S0101_C04_008E,S0101_C04_009E,S0101_C04_010E,S0101_C04_011E,S0101_C04_012E,S0101_C04_013E,S0101_C04_014E,S0101_C04_015E,S0101_C04_016E,S0101_C04_017E,S0101_C04_018E,S0101_C04_019E&in=state:*&in=county:*&for=county:*
2019 Renameing GEO_ID  =  Geography Changing type to string
2019 Renameing NAME  =  Geographic Area Name Changing type to string
2019 Renameing S0101_C04_002E  =  Under 5 years 2019 (Estimate) Changing type to float
2019 Renameing S0101_C04_003E  =  5 to 9 years 2019 (Estimate) Changing type to float
2019 Renameing S0101_C04_004E  =  10 to 14 years 2019 (Estimate) Changing type to float
2019 Renameing S0101_C04_005E  =  15 to 19 years 2019 (Estimate) Changing type to float
2019 Renameing S0101_C04_006E  =  20 to 24 years 2019 (Estimate) Changing type to float
2019 Renameing S0101_C04_007E  =  25 to 

In [52]:
acs_df['2019 Percent Male'].head()

,Geography,Geographic Area Name,Under 5 years 2019 (Estimate),5 to 9 years 2019 (Estimate),10 to 14 years 2019 (Estimate),15 to 19 years 2019 (Estimate),20 to 24 years 2019 (Estimate),25 to 29 years 2019 (Estimate),30 to 34 years 2019 (Estimate),35 to 39 years 2019 (Estimate),40 to 44 years 2019 (Estimate),45 to 49 years 2019 (Estimate),50 to 54 years 2019 (Estimate),55 to 59 years 2019 (Estimate),60 to 64 years 2019 (Estimate),65 to 69 years 2019 (Estimate),70 to 74 years 2019 (Estimate),75 to 79 years 2019 (Estimate),80 to 84 years 2019 (Estimate),85 years and over 2019 (Estimate),state,county
0,0500000US17051,"Fayette County, Illinois",5.4,5.4,5.5,5.9,7.1,7.1,6.3,7.0,6.1,6.7,7.2,7.4,6.7,4.9,4.6,3.5,1.6,1.7,17,051
1,0500000US17107,"Logan County, Illinois",5.8,5.3,5.2,7.3,7.0,6.5,6.4,7.4,6.5,6.7,6.5,7.1,6.8,5.2,3.8,2.9,1.4,2.4,17,107
2,0500000US17165,"Saline County, Illinois",6.8,5.2,6.8,6.8,5.6,6.4,6.2,7.4,3.8,6.5,6.7,6.5,7.5,6.5,4.1,3.1,2.2,1.8,17,165
3,0500000US17097,"Lake County, Illinois",5.9,7.0,7.3,8.3,7.8,5.8,5.6,6.1,6.4,6.9,7.2,7.1,6.1,4.6,3.2,2.1,1.3,1.2,17,097
4,0500000US17127,"Massac County, Illinois",5.8,7.7,5.1,5.6,5.6,5.8,5.1,5.4,6.9,6.7,6.9,7.0,7.5,4.8,5.9,3.8,2.9,1.5,17,127


In [53]:
acs_df['2019 Percent Female'].head()

,Geography,Geographic Area Name,Under 5 years 2019 (Estimate),5 to 9 years 2019 (Estimate),10 to 14 years 2019 (Estimate),15 to 19 years 2019 (Estimate),20 to 24 years 2019 (Estimate),25 to 29 years 2019 (Estimate),30 to 34 years 2019 (Estimate),35 to 39 years 2019 (Estimate),40 to 44 years 2019 (Estimate),45 to 49 years 2019 (Estimate),50 to 54 years 2019 (Estimate),55 to 59 years 2019 (Estimate),60 to 64 years 2019 (Estimate),65 to 69 years 2019 (Estimate),70 to 74 years 2019 (Estimate),75 to 79 years 2019 (Estimate),80 to 84 years 2019 (Estimate),85 years and over 2019 (Estimate),state,county
0,0500000US17051,"Fayette County, Illinois",5.6,4.9,7.0,5.6,5.5,5.9,5.7,5.8,5.3,6.2,6.3,8.2,6.2,5.3,5.7,4.7,3.1,2.9,17,051
1,0500000US17107,"Logan County, Illinois",4.9,4.9,5.3,6.6,7.2,7.2,6.5,6.3,5.4,5.7,6.0,7.6,6.1,5.7,4.4,3.1,2.0,5.1,17,107
2,0500000US17165,"Saline County, Illinois",5.7,4.7,6.4,4.8,4.7,6.7,5.9,6.4,4.7,6.2,7.2,7.2,7.4,5.7,5.7,3.7,2.8,4.1,17,165
3,0500000US17097,"Lake County, Illinois",5.7,6.4,7.2,7.3,6.5,5.2,5.5,6.1,6.6,7.1,7.5,7.3,6.5,5.0,3.5,2.4,1.7,2.3,17,097
4,0500000US17127,"Massac County, Illinois",5.4,9.5,4.3,5.1,4.9,5.0,4.8,6.0,6.1,5.7,6.5,6.2,7.8,5.8,5.4,3.6,4.1,3.6,17,127


## Step 2: Clean Data
Data cleaning is an important step in the data science process. This step is often the hardest and most time consuming. 

### 2.1. Clean ACS Data
#### 2.1.1 Make sure variable types are set correctly
For the ACS data we have both population counts and precents. The percentage estiamtes are not integers - they are "floats" which refers the the decimal place that can float in the number between the integer and the fractional part. Here is more information on Python number types https://www.tutorialspoint.com/python/python_numbers.htm 

#### 2.1.2 Use descriptive statistics to check cleaning
A descriptive statistics table is a great way to check to make sure the variables have been created correctly.

In [54]:
float_col_list = list(acs_df['2019 Percent Female'].select_dtypes(include=['float']).columns)
table2 = acs_df['2019 Percent Female'][float_col_list].describe().T
varformat = "{:,.2f}" # The variable format adds a comma and rounds up
table_title = "Table 2. Descriptive statistics for variables by county, 2019 5-year ACS."
table2 = table2.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table2

,count,mean,std,min,25%,50%,75%,max
Under 5 years 2019 (Estimate),"3,220.00",5.65,1.49,0.00,4.90,5.60,6.20,36.40
5 to 9 years 2019 (Estimate),"3,220.00",5.96,1.41,0.00,5.10,5.90,6.70,14.10
10 to 14 years 2019 (Estimate),"3,220.00",6.26,1.41,0.00,5.50,6.20,7.00,16.90
15 to 19 years 2019 (Estimate),"3,220.00",6.18,1.63,0.00,5.38,6.00,6.80,19.70
20 to 24 years 2019 (Estimate),"3,220.00",5.80,2.31,0.00,4.80,5.40,6.30,28.00
25 to 29 years 2019 (Estimate),"3,220.00",5.76,1.36,0.00,5.00,5.70,6.40,15.50
30 to 34 years 2019 (Estimate),"3,220.00",5.64,1.14,0.00,5.00,5.60,6.20,16.70
35 to 39 years 2019 (Estimate),"3,220.00",5.77,1.22,0.00,5.10,5.80,6.50,14.40
40 to 44 years 2019 (Estimate),"3,220.00",5.60,1.14,0.00,5.00,5.60,6.20,15.70
45 to 49 years 2019 (Estimate),"3,220.00",5.97,0.99,0.00,5.50,6.00,6.50,14.30


## Step 3: Merge Data together
Merging data together is a powerful tool in data science workflows. Merge data refers to combining data that represent the same unit of analysis. In this example all three datasets represent counties in the US. Each dataframe has a common variable called GEO_ID which uniquely identifies the counties.

Here is a good overview of data merging in python:
https://towardsdatascience.com/why-and-how-to-use-merge-with-pandas-in-python-548600f7e738

### 3.1 - Merge Part 1 - Combine ACS Data

In [ ]:
merge_df = pd.merge(left = acs_df['2012'],
                    right = acs_df['2019'],
                    left_on = ['Geography','Geographic Area Name'],
                    right_on = ['Geography','Geographic Area Name'],
                    how = 'inner')
merge_df.head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,tract_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),state_y,county_y,tract_y
0,1400000US48113014132,"Census Tract 141.32, Dallas County, Texas",1154,162,0.8,1.4,48,113,014132,1293,165,0.9,1.3,48,113,014132
1,1400000US48113014133,"Census Tract 141.33, Dallas County, Texas",2610,340,4.4,3.3,48,113,014133,3442,297,2.9,1.8,48,113,014133
2,1400000US48113014134,"Census Tract 141.34, Dallas County, Texas",1968,182,1.5,1.8,48,113,014134,2239,178,0.5,0.8,48,113,014134
3,1400000US48113014135,"Census Tract 141.35, Dallas County, Texas",2201,224,0.6,0.9,48,113,014135,2506,193,0.0,1.7,48,113,014135
4,1400000US48113014136,"Census Tract 141.36, Dallas County, Texas",3731,316,2.5,1.7,48,113,014136,3908,318,1.7,2.3,48,113,014136


### 3.3.1 Remove repeated columns

In [ ]:
## The state and county columns also match - we can drop the repeated columns
merge_df = merge_df.drop(['state_y','county_y'], axis = 1)
merge_df.head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,tract_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),tract_y
0,1400000US48113014132,"Census Tract 141.32, Dallas County, Texas",1154,162,0.8,1.4,48,113,014132,1293,165,0.9,1.3,014132
1,1400000US48113014133,"Census Tract 141.33, Dallas County, Texas",2610,340,4.4,3.3,48,113,014133,3442,297,2.9,1.8,014133
2,1400000US48113014134,"Census Tract 141.34, Dallas County, Texas",1968,182,1.5,1.8,48,113,014134,2239,178,0.5,0.8,014134
3,1400000US48113014135,"Census Tract 141.35, Dallas County, Texas",2201,224,0.6,0.9,48,113,014135,2506,193,0.0,1.7,014135
4,1400000US48113014136,"Census Tract 141.36, Dallas County, Texas",3731,316,2.5,1.7,48,113,014136,3908,318,1.7,2.3,014136


## Step 4: Explore Data
### 4.1 - Which Census Geographies had significant changes?

In [ ]:
merge_df = merge_df.sort_values(by=['Difference between Percents'], ascending=False)
merge_df.loc[(merge_df['Workers 16 years and over 2012 (Estimate)'] > 4000) &
             (merge_df['Difference between Percents'] > 0) &
             (merge_df['county_x'] == '201') &
             (merge_df['Test Statistic'] >= 3)]

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,tract_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),tract_y,Difference between Percents,Standard Error 1,Standard Error 2,Difference Standard Error,Test Statistic
1312,1400000US48201314002,"Census Tract 3140.02, Harris County, Texas",4328,541,16.8,5.1,48,201,314002,4247,571,28.4,7.9,314002,11.6,4.802432,3.100304,0.174941,66.308216
4808,1400000US48201452700,"Census Tract 4527, Harris County, Texas",4968,660,1.5,1.3,48,201,452700,6260,649,6.6,3.6,452700,5.1,2.188450,0.790274,0.429781,11.866512
3840,1400000US48201452100,"Census Tract 4521, Harris County, Texas",4398,626,0.8,0.9,48,201,452100,5718,939,5.8,5.6,452100,5.0,3.404255,0.547112,0.290028,17.239697
4494,1400000US48201533400,"Census Tract 5334, Harris County, Texas",4135,531,0.0,0.9,48,201,533400,4596,648,3.9,2.5,533400,3.9,1.519757,0.547112,0.619104,6.299427
2022,1400000US48201541003,"Census Tract 5410.03, Harris County, Texas",7058,425,0.2,0.4,48,201,541003,7033,818,3.1,2.1,541003,2.9,1.276596,0.243161,0.769499,3.768688
5224,1400000US48201510900,"Census Tract 5109, Harris County, Texas",4145,516,1.0,1.6,48,201,510900,5475,532,3.3,2.9,510900,2.3,1.762918,0.972644,0.496664,4.630898
2212,1400000US48201240902,"Census Tract 2409.02, Harris County, Texas",5359,599,0.5,0.7,48,201,240902,7826,726,2.7,2.7,240902,2.2,1.641337,0.425532,0.589761,3.730324
958,1400000US48201552800,"Census Tract 5528, Harris County, Texas",4208,356,2.4,1.7,48,201,552800,4950,574,4.5,4.2,552800,2.1,2.553191,1.033435,0.363054,5.784261
